# TP1 Alejandro balladares 101118

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Punto 6

6 44. Si decimos que la ubicación de un usuario es el promedio de la latitud y longitud de
los contenidos geolocalizados para los cuales editó la última versión (ignorar
usuarios que no editaron contenido geolocalizado). ¿Cuáles son los dos usuarios
más cercanos? (⭐⭐⭐)

In [4]:
id_contents = '1SarxhT65lTj5vztK-QnvrO625qMjCokB'

downloaded = drive.CreateFile({'id': id_contents})
downloaded.GetContentFile('contents.csv')

df_contents = pd.read_csv('contents.csv')

In [5]:
id_geo_tags = '1EwtdzLu_QXCifu85wH-memXQtX5118lT'
downloaded = drive.CreateFile({'id': id_geo_tags})
downloaded.GetContentFile('geo_tags.csv')

df_geo_tags = pd.read_csv('geo_tags.csv')

In [6]:
df_mean = df_geo_tags.groupby('gt_page_id')['gt_lat', 'gt_lon'].agg('mean')
df_mean = df_mean.reset_index()
df_mean.columns =['id', 'gt_lat', 'gt_lon']

df_merge_6 = pd.merge(df_contents, df_mean,  on="id", how="inner", sort=False)
df_merge_6 = df_merge_6.drop(columns=['namespace','revision_timestamp','revisor_comment','revision_id','parent_revision'])


In [7]:
df_merge_6['revisor_id'] = df_merge_6['revisor_id'].fillna(df_merge_6['revisor_ip'])
df_merge_6

,title,id,revisor_username,revisor_id,revisor_ip,gt_lat,gt_lon
0,Andorra,7,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,42.500000,1.500000
1,Argentina,10,Traitrich,4.64228e+06,NaN,-34.599722,-58.381944
2,Geografía de Andorra,15,Lojwe,5.33851e+06,NaN,42.506111,1.522222
3,Anoeta,26,InternetArchiveBot,4.70485e+06,NaN,43.162222,-2.070000
4,Afganistán,39,Traitrich,4.64228e+06,NaN,34.532778,69.165833
...,...,...,...,...,...,...,...
370728,Kumarghat,10010550,Carlukas,86975,NaN,24.158333,92.029722
370729,Río Makarora,10010559,Lorito987,5.95249e+06,NaN,-44.316667,169.166667
370730,Panisagar,10010567,Carlukas,86975,NaN,24.250000,92.140000
370731,Dewanpasa,10010581,Carlukas,86975,NaN,24.360639,92.139850


In [8]:
df_knn = df_merge_6.groupby('revisor_id')['gt_lat','gt_lon'].agg('mean')
#df_knn #promedio de la latitud y longitud por usuario

df_knn_aux = df_knn.reset_index()

In [9]:
buscador = NearestNeighbors()
buscador.fit(df_knn)
df_respuesta_6 = df_knn.copy()

for i,fila in df_knn.iterrows():
  tupla = buscador.kneighbors([fila], n_neighbors=2)
  
  usuario_mas_cercano = df_knn_aux.iloc[tupla[1][0][1]]['revisor_id']
  
  df_respuesta_6.loc[i, 'Usuario_mas_cercano'] = usuario_mas_cercano
  df_respuesta_6.loc[i, 'Diff_distancia'] =  tupla[0][0][1]

df_respuesta_6.head()

,gt_lat,gt_lon,Usuario_mas_cercano,Diff_distancia
revisor_id,,,,
639.0,31.458056,-47.788923,5.95253e+06,0.309989
1299.0,38.742859,-5.325884,81.9.236.76,0.190238
1430.0,41.538595,-1.630543,6.15947e+06,0.006962
1640.0,27.209055,-13.186873,5.95911e+06,0.596555
1854.0,38.412029,-0.988330,84.78.251.98,0.018550


In [10]:
df_respuesta_6 = df_respuesta_6.sort_values(by=['Diff_distancia'])
#df_respuesta_6.loc[df_respuesta_6.Diff_distancia > 0]
df_respuesta_6 #los 2 usuarios más cercanos, aunque haya muchos que tengan distancia 0

,gt_lat,gt_lon,Usuario_mas_cercano,Diff_distancia
revisor_id,,,,
142.126.195.230,-33.625356,-71.271164,154.118.191.234,0.000000
4999877.0,-34.601806,-58.399444,181.23.198.93,0.000000
148.101.28.31,18.810000,-71.230000,6.0072e+06,0.000000
148.0.14.29,18.542500,-72.338611,2806:2F0:6020:C5C3:799E:344C:3449:64FD,0.000000
146.158.238.119,49.000000,4.500000,146.158.238.119,0.000000
...,...,...,...,...
77.204.197.86,-35.200000,-150.666667,6.276e+06,18.794008
191.116.194.166,-44.000000,-176.583333,2806:2F0:90A0:C0A6:6815:A473:1F43:9021,21.037372
4068869.0,-13.937000,208.186837,186.159.192.103,29.489264


In [11]:
df_aux = df_respuesta_6.reset_index()
df_aux.loc[df_aux.revisor_id == df_aux.Usuario_mas_cercano] #no debería haber nadie que tenga como más cercano a si mismo pero son solo 87 asi que puedo ignorarlo?

,revisor_id,gt_lat,gt_lon,Usuario_mas_cercano,Diff_distancia
4,146.158.238.119,49.000000,4.500000,146.158.238.119,0.0
7,80.33.108.200,43.556111,-5.922222,80.33.108.200,0.0
10,190.52.192.132,-3.326900,-79.804900,190.52.192.132,0.0
11,2800:810:544:8546:8859:88BE:D343:325F,-34.294900,-59.320500,2800:810:544:8546:8859:88BE:D343:325F,0.0
13,4.86932e+06,-34.670278,-58.370833,4.86932e+06,0.0
...,...,...,...,...,...
171,46.193.135.36,37.975278,23.736944,46.193.135.36,0.0
173,5.6429e+06,22.767000,108.388000,5.6429e+06,0.0
174,186.34.39.241,-33.390739,-70.711211,186.34.39.241,0.0
180,83.165.239.78,45.466944,9.190000,83.165.239.78,0.0
